# CASDA public data release

Export data from the WALLABY database in the appropriate format for the public data release.

## Connect

In [1]:
import sys
sys.path.append('/mnt/shared/home/ashen/development/WALLABY_data_access/wallaby_data_access')

In [2]:
import wallaby

wallaby.connect()
wallaby.print_tags()

"Norma DR1"         	Norma DR1 data release
"NGC 5044 DR1"      	NGC 5044 DR1 data release
"Component"         	Component of a galaxy or object
"Multiplet"         	Two or more galaxies merged into a single detection.
"Hydra DR1"         	Hydra DR1 data release
"Hydra DR2"         	Hydra DR2 data release
"NGC 4636 DR1"      	NGC 4636 DR1 data release


---

## Prepare table data

Create `astropy` table for catalogue information. Set the name of the public data release.

In [21]:
# set public data release name

deposit_name = 'PublicDR1'

In [4]:
# get tables

norma_dr1_table = wallaby.get_catalog('Norma DR1')
hydra_dr1_table = wallaby.get_catalog('Hydra DR1')
hydra_dr2_table = wallaby.get_catalog('Hydra DR2')
NGC4636_dr1_table = wallaby.get_catalog('NGC 4636 DR1')

In [5]:
# join tables above

from astropy.table import join, Table

table = join(norma_dr1_table, hydra_dr1_table, join_type='outer')
table = join(table, hydra_dr2_table, join_type='outer')
table = join(table, NGC4636_dr1_table, join_type='outer')

In [6]:
# Modify table as required (add kinematic output flag)

table.rename_column('flag', 'qflag')
table['kflag'] = [0] * len(table)

---
## Add kinematic model flags

Manually update `kflag` property in table where appropriate

In [10]:
# read kinematic model flags

from astropy.io import fits

kin_models_file = 'AllTR_KinModels_kflag.fits'
hdul = fits.open(kin_models_file)

In [8]:
kflags = hdul[1].data

In [14]:
names = list(table['name'])

In [17]:
# set flags

names = list(table['name'])
for row in kflags:
    source_name, _, _, _, _, _, kflag_val = row
    source_index = names.index(source_name)
    table['kflag'][source_index] = kflag_val

---

## Create release files

Once the metadata has been added from above we can create the deposit

In [24]:
table_copy = table[:50]

In [31]:
# Export deposit data locally

wallaby.casda_deposit(table, deposit_name)

---

## Add header cards

Manually add SBIDs to header card for products after they have been created.

In [32]:
# SBIDs

sbids = {
    'Hydra': '10609,10269,10612,10626',
    'Norma': '10809,10812,10736',
    'NGC_4636': '12193,12209'
}

In [33]:
# Get all product files

import glob
product_files = glob.glob(f'{deposit_name}/**/*.fits', recursive=True)

In [37]:
# Update fits headers in product files

from astropy.io import fits
for f in product_files:
    hdul = fits.open(f, 'update')
    hdr = hdul[0].header
    if 'Hydra' in f:
        hdr['SBID'] = sbids['Hydra']
    elif 'Norma' in f:
        hdr['SBID'] = sbids['Norma']
    elif 'NGC_4636' in f:
        hdr['SBID'] = sbids['NGC_4636']
    else:
        raise Exception('File does not have a corresponding team release tag')
    hdul.close()


---

# CASDA deposit function debugging

In [ ]:
import csv
import os
from astropy.io.votable import from_table, writeto
from astropy.io.votable.tree import Param

table_columns = {}
with open('/mnt/shared/home/ashen/development/WALLABY_data_access/wallaby_data_access/source_column_metadata.csv', 'r') as file:
    reader = csv.reader(file)
    for row in reader:
        name, ucd, units, description = row
        units = units.replace('pixel', 'pix')
        if units == '-':
            units = None
        table_columns[name] = (ucd, units, description)

column_names = [k for k in table_columns.keys()]

In [ ]:
# Export catalogue
table_copy.remove_column('id')
table_copy.remove_column('tags')
table_copy.remove_column('comments')
table_copy = table_copy[column_names]
votable = from_table(table_copy)
votable.version = '1.3'

In [ ]:
fields = []
for name, meta in table_columns.items():
    ucd, units, description = meta
    field = votable.get_field_by_id(name)
    field.ucd = ucd
    field.unit = units
    field.description = description
    fields.append(field)

In [ ]:
# TODO(austin): reorder astropy table columns to match headers from fields ^

for resource in votable.resources:        
    # update rows
    for t in resource.tables:
        for i, row in enumerate(fields):
            t.fields[i] = row

    # add params
    resource.params.append(Param(votable, ID='catalogueName', name='Catalogue Name', value=deposit_name, arraysize='59'))
    resource.params.append(Param(votable, ID='indexedFields', name='Indexed Fields', value='name,ra,dec,freq,f_sum,w20,team_release', arraysize='255'))
    resource.params.append(Param(votable, ID='principalFields', name='Principal Fields', value='name,ra,dec,freq,f_sum,w20,team_release', arraysize='255'))

    # rename columns
    # TODO(austin): rename source finding flag to "qflag"
    # TODO(austin): add "kflag" column

catalogue_file = 'catalogue.xml'
writeto(votable, catalogue_file)